In [36]:
from queue import Queue
from unidecode import unidecode
import networkx as nx
import spotipy #https://spotipy.readthedocs.io/en/latest/
from spotipy.oauth2 import SpotifyClientCredentials
import statistics 

In [37]:
# Charles profile link:
# https://open.spotify.com/user/12167056236?si=BfbG8v9wRsOTnilpjP9ibg
# spotify:user:12167056236:playlist:1ahkvZq6vSkYCmqHiPpLJn
charles_id = '12167056236'

# Jazz playlist link:
# https://open.spotify.com/user/12167056236/playlist/2ek4gj4PDBF4aspnYWfBmC?si=eq_YK6V8SdKiuhu1zj5VPA
# spotify:user:12167056236:playlist:2ek4gj4PDBF4aspnYWfBmC
jazz_id = '2ek4gj4PDBF4aspnYWfBmC'

# Bossa nova playlist link:
# https://open.spotify.com/user/12167056236/playlist/1ahkvZq6vSkYCmqHiPpLJn?si=DQT4XTRYSwWagYejOs34Ew
bossanova_id = '1ahkvZq6vSkYCmqHiPpLJn'

In [38]:
# client_credentials_manager = SpotifyClientCredentials(client_id='943e4795c796489f9243c6e7c9f64fc7', client_secret='c6c980434cb74acc9a7408e5c99618b1')
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
# spotify:user:12167056236:playlist:2ek4gj4PDBF4aspnYWfBmC

In [39]:
# playlists = sp.user_playlists('spotify')
# tracks = user_playlist_tracks(user = 12167056236, playlist_id=2ek4gj4PDBF4aspnYWfBmC?si=j-MSydgGR1O3KgIUea6Yzw)

In [40]:
client_credentials_manager = SpotifyClientCredentials(client_id='943e4795c796489f9243c6e7c9f64fc7', client_secret='c6c980434cb74acc9a7408e5c99618b1')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# def calc_feat_dict(results):
#     artists_musics = {}
#     for i, item in enumerate(results['items']):
#         track = item['track']
#         artists_musics[track['artists'][0]['name']] = (track['name'].split(" - ")[0])
#         #print(track['id'])
#         #print("%d: %s - %s" % (i, track['artists'][0]['name'], track['name'].split(" - ")[0]))
#     return artists_musics

def calc_tuple(results):
    artists_musics = []
    for i, item in enumerate(results['items']):
        track = item['track']
        artists_musics.append((track['artists'][0]['id'], track['artists'][0]['name'], track['id'], track['name']))
        #print((track['artists'][0]['name'], (track['name'].split(" - ")[0])))
    return artists_musics

In [41]:
def art_mus_dict(playlist_id):

    '''
    Cria o dicionário: {artista: [músicas]}
    '''
    art_mus = {}

    results = sp.user_playlist(user = charles_id, playlist_id = playlist_id, fields = "tracks, next")
    tracks = results['tracks']

    while tracks['next']: #passa pelos 10 conjuntos de tracks
        tracks = sp.next(tracks)
        data = calc_tuple(tracks)    

        #inicializa o dicio com o primeiro artista deste cojunto
        for i in range(len(data)): #rodará 100 vezes
            if data[i][0] not in art_mus.keys(): #trocou artista
                art_mus[data[i][0]] = [data[i][2]]
            else: #mesmo artista
                art_mus[data[i][0]].append(data[i][2])
    return art_mus

In [42]:
def art_id_dict(playlist_id):
    '''
    Gera um dicionário: {artista: id do artista}
    '''

    id_art = {}

    results = sp.user_playlist(user = charles_id, playlist_id = playlist_id, fields = "tracks, next")
    tracks = results['tracks']

    while tracks['next']: #passa pelos 10 conjuntos de tracks
        tracks = sp.next(tracks)
        data = calc_tuple(tracks)    

        #inicializa o dicio com o primeiro artista deste cojunto
        for i in range(len(data)): #rodará 100 vezes
            id_art[data[i][0]] = [data[i][1]]
    return id_art

In [43]:
def art_feats(art_mus):
    '''
    Gera um dicionário: {id do artista: {feats das músicas}}
    '''

    # https://developer.spotify.com/documentation/web-api/reference/object-model/#audio-features-object
    art_feats = {}

    # loop para obter as 'features' das músicas e adicionar à um novo dicio
    for artist, songs in art_mus.items():
        if (len(songs) > 49): # limitação na api do spotify de 'track features' é de 50 músicas
            for song in songs:
                if artist not in art_feats.keys():
                    art_feats[artist] = [sp.audio_features(song)[0]]
                else:
                    art_feats[artist].append(sp.audio_features(song)[0])
        else:
            art_feats[artist] = sp.audio_features(songs)
            feats = sp.audio_features(songs)
    return art_feats

In [44]:
def mus_feats_dict(art_feats):
    '''
    Gera um dicionário: {id do artista: [feats médias]}
    '''

    art_mus_feats = {}
    #  loop para obter as médias das 'features' das músicas de um artista
    for artist, feats in art_feats.items():
    #     print(type(feats))
    #     print(artist)
    #     for i in feats:
    #         print(i)
        art_mus_feats[artist] = {
                'danceability': statistics.mean([x.get('danceability') for x in feats]),
                'energy': statistics.mean([x.get('energy') for x in feats]),
                'key': statistics.mean([x.get('key') for x in feats]),
                'loudness': statistics.mean([x.get('loudness') for x in feats]),
                'mode': statistics.mean([x.get('mode') for x in feats]),
                'speechiness': statistics.mean([x.get('speechiness') for x in feats]),
                'acousticness': statistics.mean([x.get('acousticness') for x in feats]),
                'instrumentalness': statistics.mean([x.get('instrumentalness') for x in feats]),
                'liveness': statistics.mean([x.get('liveness') for x in feats]),
                'valence': statistics.mean([x.get('valence') for x in feats]),
                'tempo': statistics.mean([x.get('tempo') for x in feats]),
    #             '': statistics.mean([x.get('key') for x in feats]),
    #             '': statistics.mean([x.get('key') for x in feats]),
            }
    return art_mus_feats

In [45]:
def genre_feats(art_mus_feats, genre):
    '''
    Gera um dicionário: {Nome do gênero: [feats médias]}
    '''

    genre_feats = {}
    #  loop para obter as médias das 'features' das músicas de um artista
    genre_feats[genre] = {
        'danceability': statistics.mean([x.get('danceability') for key,x in art_mus_feats.items()]),
        'energy': statistics.mean([x.get('energy') for key,x in art_mus_feats.items()]),
        'key': statistics.mean([x.get('key') for key,x in art_mus_feats.items()]),
        'loudness': statistics.mean([x.get('loudness') for key,x in art_mus_feats.items()]),
        'mode': statistics.mean([x.get('mode') for key,x in art_mus_feats.items()]),
        'speechiness': statistics.mean([x.get('speechiness') for key,x in art_mus_feats.items()]),
        'acousticness': statistics.mean([x.get('acousticness') for key,x in art_mus_feats.items()]),
        'instrumentalness': statistics.mean([x.get('instrumentalness') for key,x in art_mus_feats.items()]),
        'liveness': statistics.mean([x.get('liveness') for key,x in art_mus_feats.items()]),
        'valence': statistics.mean([x.get('valence') for key,x in art_mus_feats.items()]),
        'tempo': statistics.mean([x.get('tempo') for key,x in art_mus_feats.items()])
    }

    return genre_feats

In [46]:
jazz_art_mus = art_mus_dict(jazz_id)
bossa_art_mus = art_mus_dict(bossanova_id)

In [47]:
jazz_art_id = art_id_dict(jazz_id)
bossa_art_id = art_id_dict(bossanova_id)

In [48]:
jazz_art_feats = art_feats(jazz_art_mus)
bossa_art_feats = art_feats(bossa_art_mus)

In [49]:
jazz_art_mus_feats = mus_feats_dict(jazz_art_feats)
bossa_art_mus_feats = mus_feats_dict(bossa_art_feats)

In [51]:
lista = ['acousticness',  'danceability',  'energy',  'instrumentalness',  'key',  'liveness',  'loudness',  'speechiness',  'valence']
extremos_jazz = []
for i in lista:
    jazz_max = max([x.get(i) for key,x in jazz_art_mus_feats.items()])
    jazz_min = min([x.get(i) for key,x in jazz_art_mus_feats.items()])

    for key, x in jazz_art_mus_feats.items():
        if (x.get(i) == jazz_max):
            extremos_jazz.append((i, key, jazz_max, "max"))
        if (x.get(i) == jazz_min):
            extremos_jazz.append((i, key, jazz_min, "min"))
            
extremos_bossa = []
for i in lista:
    bossa_max = max([x.get(i) for key,x in bossa_art_mus_feats.items()])
    bossa_min = min([x.get(i) for key,x in bossa_art_mus_feats.items()])

    for key, x in bossa_art_mus_feats.items():
        if (x.get(i) == bossa_max):
            extremos_bossa.append((i, key, bossa_max, "max"))
        if (x.get(i) == bossa_min):
            extremos_bossa.append((i, key, bossa_min, "min"))
            
print(extremos_bossa)
print(extremos_jazz)

[('acousticness', '5M1PEUHGPirQtE9SjKLrMH', 0.959, 'max'), ('acousticness', '7zsxdMsODmHKTbTB00t9wS', 0.0312, 'min'), ('danceability', '5M1PEUHGPirQtE9SjKLrMH', 0.159, 'min'), ('danceability', '04dFhs355aJPkUXV9CwM3L', 0.776, 'max'), ('energy', '5RYHqDtvrm4QLsvcoOHmR9', 0.839, 'max'), ('energy', '2EZ5KFNGsDnn9PYgqWqn73', 0.185, 'min'), ('instrumentalness', '1j081CNX8swtqfQCXGUc4v', 0, 'min'), ('instrumentalness', '7ixbHztB2Y1LUtvCTCL9MZ', 0, 'min'), ('instrumentalness', '4cn4gMq0KXORHeYA45PcBi', 0, 'min'), ('instrumentalness', '4KtH5Ery1JspML3ECanPmZ', 0.959, 'max'), ('instrumentalness', '6aewGwsqlu1cmev2xBJoht', 0, 'min'), ('instrumentalness', '03zMprDSi8xGJbXYayx6ly', 0, 'min'), ('key', '0cWAA0nMjRXh8o61j3hUgm', 11, 'max'), ('key', '39H7b1PRozlNLdkWZhWG40', 0, 'min'), ('key', '5yrpqAQx2OwR62pNE7d5bm', 0, 'min'), ('key', '3cjV65izDdMmWE2LVwv2rb', 0, 'min'), ('key', '6LjzVtpQrLrjcTRgEwI2DM', 0, 'min'), ('liveness', '04dFhs355aJPkUXV9CwM3L', 0.0603, 'min'), ('liveness', '6O67GI6ayhWHyFm

In [53]:
jazz_art_mus_feats["6pVkGhtBvjj1P52yn0tNLG"]

{'acousticness': 0.5435,
 'danceability': 0.8105,
 'energy': 0.548,
 'instrumentalness': 0.837,
 'key': 10,
 'liveness': 0.34199999999999997,
 'loudness': -12.027999999999999,
 'mode': 1,
 'speechiness': 0.04625,
 'tempo': 130.2375,
 'valence': 0.9644999999999999}

In [45]:
jazz_feats = genre_feats(jazz_art_mus_feats, 'Jazz')
bossa_feats = genre_feats(bossa_art_mus_feats, 'Bossa Nova')

In [ ]:
#tupla com o id que tem valor maximo e o id que tem valor mínimo. 
#Para key e instrumentalness obtivemos diversas ocorrências do valor mínimo, por isso pegamos apenas uma das ocorrências do valor mínimo para cada um desses indicadores.
combinacoes = [("5M1PEUHGPirQtE9SjKLrMH","5xdQf8ZY4CFEtt0LKztj32"),
               ("0DPDe1cl41tscTH0TE8Elu", "7zsxdMsODmHKTbTB00t9wS"),
               ("04dFhs355aJPkUXV9CwM3L", "273EXbTI4AKNTHjpdLOhRQ"),
               ("6pVkGhtBvjj1P52yn0tNLG", "5M1PEUHGPirQtE9SjKLrMH"),
               ("5RYHqDtvrm4QLsvcoOHmR9", "7uFjZms2FHdipWrsQFljwt"),
               ("2LtqGCRMKbgeuUnq6OOOeP", "2EZ5KFNGsDnn9PYgqWqn73"),
               ("4KtH5Ery1JspML3ECanPmZ", "273EXbTI4AKNTHjpdLOhRQ"),
               ("6BufIg68WvvzrEqoDI19Kl", "1j081CNX8swtqfQCXGUc4v"),
               ("0cWAA0nMjRXh8o61j3hUgm", "5cbutZUQE7SUCA6MsEMbBv"),
               ("4F7Q5NV6h5TSwCainz8S5A", "39H7b1PRozlNLdkWZhWG40"),
               ("6O67GI6ayhWHyFmOOJRLEq", "2oVwztjpHpJlAvlVVuqVa0"),
               ("4iRZAbYvBqnxrbs6K25aJ7", "04dFhs355aJPkUXV9CwM3L"),
               ("03zMprDSi8xGJbXYayx6ly", ""),
               ("", "4KtH5Ery1JspML3ECanPmZ"),
               ("3LokxmaXXZpL7sLZASDzCS", ""),
               ("", "6mpinUbWcol0zWCAy6XLXu"),
               ("04dFhs355aJPkUXV9CwM3L", ""),
               ("", "2EZ5KFNGsDnn9PYgqWqn73")]